In [1]:
import pandas as pd

In [7]:
data = pd.read_csv("/projectnb/cs640grp/materials/UBC-OCEAN_CS640/test.csv")

In [8]:
data.head()

,image_id,label
0,7482,CC


In [2]:
# data = pd.("/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80/10077.jpg")

In [3]:
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
Image.MAX_IMAGE_PIXELS = None 

In [5]:
img = Image.open("/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80/10077.jpg")

In [6]:
# img

In [7]:
# plt.imshow(img)
# plt.axis('off') # to turn off axis
# plt.show()

In [11]:
!pip install torch 


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/96/82/0966469ded5946cb4c18dd11b04eac78c943269fc79d290740d6477005e8/torch-2.1.1-cp310-cp310-manylinux1_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 203.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 262.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 207.0 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for nvidia-cudnn-cu12==8.9.2.26 from https://files.pythonhosted.org/packages/ff/74/a2e2be7fb83aaedec84f391f082cf765dfb635e7caa9b49065f73e4835d8/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 247.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 254.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━

In [14]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.5 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

# Define the dataset
# class CancerSlideDataset(Dataset):
class CancerSlideDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        # Define a mapping for the labels
        self.label_mapping = {'CC': 0, 'MC': 1, 'HGSC': 2, 'LGSC': 3, 'EC': 4}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0]))
        img_name+=".jpg"
        image = Image.open(img_name)
        label_str = self.annotations.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        # Map the string label to an integer
        label = self.label_mapping[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, label


#     def __getitem__(self, index):
#         img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0]))
#         img_name+=".jpg"
#         image = Image.open(img_name)
#         label = self.annotations.iloc[index, 1]

#         if self.transform:
#             image = self.transform(image)

#         return image, label
    
#     def __getitem__(self, index):
#         img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0]))
#         img_name+=".jpg"
#         image = Image.open(img_name)
#         label = self.annotations.iloc[index, 1]

#         if self.transform:
#             image = self.transform(image)

#         label = torch.tensor(label, dtype=torch.long)  # Convert label to a long Tensor

#         return image, label
    
#     def __getitem__(self, index):
#         img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0]))
#         img_name+=".jpg"
#         image = Image.open(img_name)
#         label = self.annotations.iloc[index, 1]

#         if self.transform:
#             image = self.transform(image)

#         # Ensure label is an integer before conversion to Tensor
#         label = int(label)  
#         label = torch.tensor(label, dtype=torch.long)  # Convert label to a long Tensor

#         return image, label



# Define the model
class CancerClassifier(nn.Module):
    def __init__(self):
        super(CancerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 50 * 50, 512)
        self.fc2 = nn.Linear(512, 5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 50 * 50)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the data
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

train_dataset = CancerSlideDataset(csv_file='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train.csv', root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Instantiate the model
model = CancerClassifier()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in tqdm(range(10)):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')


  0%|          | 0/10 [01:02<?, ?it/s]


KeyboardInterrupt: 

In [29]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm  # assuming tqdm is installed, otherwise remove this

class CancerSlideDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, limit=None):
        self.annotations = pd.read_csv(csv_file)
        if limit is not None:
            self.annotations = self.annotations[:limit]
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = {'CC': 0, 'MC': 1, 'HGSC': 2, 'LGSC': 3, 'EC': 4}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0])) + ".jpg"
        image = Image.open(img_name)
        label_str = self.annotations.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        label = self.label_mapping[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, label

# Other parts of your code (model definition, etc.) remain unchanged

# Load the data with a limit of 10 images
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

train_dataset = CancerSlideDataset(csv_file='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train.csv', 
                                   root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                   transform=transform, 
                                   limit=10)  # Limiting to 10 images
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

model = CancerClassifier()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in tqdm(range(10)):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')


 10%|█         | 1/10 [00:24<03:44, 24.93s/it]

Epoch 1, Loss: 10.530348896980286


 20%|██        | 2/10 [00:50<03:22, 25.34s/it]

Epoch 2, Loss: 4.859254837036133


 30%|███       | 3/10 [01:16<02:58, 25.48s/it]

...

Epoch 4, Loss: 1.0461580753326416


 50%|█████     | 5/10 [02:07<02:08, 25.63s/it]

Epoch 5, Loss: 0.5421909590562185


 50%|█████     | 5/10 [02:13<02:13, 26.72s/it]


KeyboardInterrupt: 

In [31]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm  # assuming tqdm is installed, otherwise remove this

class CancerSlideDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, limit=None):
        self.annotations = pd.read_csv(csv_file)
        if limit is not None:
            self.annotations = self.annotations[:limit]
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = {'CC': 0, 'MC': 1, 'HGSC': 2, 'LGSC': 3, 'EC': 4}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0])) + ".jpg"
        image = Image.open(img_name)
        label_str = self.annotations.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        label = self.label_mapping[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, label

# Other parts of your code (model definition, etc.) remain unchanged



transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

train_dataset = CancerSlideDataset(csv_file='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train.csv', 
                                   root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                   transform=transform, 
                                   limit=100)  # Limiting to 10 images
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

model = CancerClassifier()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in tqdm(range(10)):  # loop over the dataset multiple times
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Accuracy: {accuracy}%')

print('Finished Training')

 10%|█         | 1/10 [04:39<41:51, 279.07s/it]

Epoch 1, Loss: 3.9572401905059813, Accuracy: 33.0%


 20%|██        | 2/10 [09:11<36:40, 275.07s/it]

Epoch 2, Loss: 1.5262813997268676, Accuracy: 32.0%


 30%|███       | 3/10 [13:44<31:58, 274.12s/it]

Epoch 3, Loss: 1.5476440286636353, Accuracy: 30.0%


 30%|███       | 3/10 [14:00<32:40, 280.13s/it]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm

class CancerSlideDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, limit=None):
        self.annotations = pd.read_csv(csv_file)
        if limit is not None:
            self.annotations = self.annotations[:limit]
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = {'CC': 0, 'MC': 1, 'HGSC': 2, 'LGSC': 3, 'EC': 4}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0])) + ".jpg"
        image = Image.open(img_name)
        label_str = self.annotations.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        label = self.label_mapping[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, label

class CancerClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super(CancerClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.model(x)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CancerSlideDataset(csv_file='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train.csv', 
                                   root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                   transform=transform, 
                                   limit=100)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

model = CancerClassifier()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(10)):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    accuracy = 100 * correct_predictions / total_predictions
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Accuracy: {accuracy}%')

print('Finished Training')


/usr4/cs640/gaurav57/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr4/cs640/gaurav57/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
 10%|█         | 1/10 [04:22<39:23, 262.56s/it]

Epoch 1, Loss: 2.2039502477645874, Accuracy: 28.0%


 20%|██        | 2/10 [08:45<35:03, 262.98s/it]

Epoch 2, Loss: 1.4885120177268982, Accuracy: 35.0%


In [27]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm

# Define the dataset
class CancerSlideDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, limit=None):
        self.annotations = pd.read_csv(csv_file)
        if limit is not None:
            self.annotations = self.annotations[:limit]
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = {'CC': 0, 'MC': 1, 'HGSC': 2, 'LGSC': 3, 'EC': 4}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0])) + ".jpg"
        image = Image.open(img_name)
        label_str = self.annotations.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        label = self.label_mapping[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, label

# Define the model
class CancerClassifier(nn.Module):
    def __init__(self):
        super(CancerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 50 * 50, 512)
        self.fc2 = nn.Linear(512, 5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 50 * 50)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Image transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

# Load the datasets with a limit of 10 images for training
train_dataset = CancerSlideDataset(csv_file='train_subset.csv', 
                                   root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                   transform=transform, 
                                   limit=10)
val_dataset = CancerSlideDataset(csv_file='val_subset.csv', 
                                 root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                 transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

model = CancerClassifier()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training and validation loop with accuracy calculation
for epoch in tqdm(range(10)):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()  # Set the model to training mode
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    training_accuracy = correct / total

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No gradient calculation during validation
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total

    print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}, Training Accuracy: {training_accuracy}, Validation Loss: {val_loss /



SyntaxError: unterminated string literal (detected at line 119) (474830655.py, line 119)

In [28]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm

# Define the dataset
class CancerSlideDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, limit=None):
        self.annotations = pd.read_csv(csv_file)
        if limit is not None:
            self.annotations = self.annotations[:limit]
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = {'CC': 0, 'MC': 1, 'HGSC': 2, 'LGSC': 3, 'EC': 4}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0])) + ".jpg"
        image = Image.open(img_name)
        label_str = self.annotations.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        label = self.label_mapping[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, label

# Define the model
class CancerClassifier(nn.Module):
    def __init__(self):
        super(CancerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 50 * 50, 512)
        self.fc2 = nn.Linear(512, 5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 50 * 50)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Image transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

# Load the datasets with a limit of 10 images for training
train_dataset = CancerSlideDataset(csv_file='train_subset.csv', 
                                   root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                   transform=transform, 
                                   limit=10)
val_dataset = CancerSlideDataset(csv_file='val_subset.csv', 
                                 root_dir='/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80', 
                                 transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

model = CancerClassifier()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training and validation loop with accuracy calculation
for epoch in tqdm(range(10)):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()  # Set the model to training mode
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    training_accuracy = correct / total

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No gradient calculation during validation
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total

    print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}, Training Accuracy: {training_accuracy}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {val_accuracy}')

print('Finished Training')


  0%|          | 0/10 [03:19<?, ?it/s]


KeyboardInterrupt: 

In [21]:
from tqdm import tqdm 

In [9]:
pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
